In [106]:
import numpy as np
import numpy.ma as ma
# import krippendorff
import rpy2
%load_ext rpy2.ipython

In [109]:
folder = "data/tese"

In [1]:
## Cleaning database
last_id = 132
problems = Problem.objects.filter(id__gt=last_id)
# solutions_obj = Solution.objects.filter(problem__in=problems).update(ignore=True)
print("Problems to be ignored: %d" % problems.count())

problems = Problem.objects.filter(id__lte=last_id)
# problems = Problem.objects.all()
print("Problems to be used: %d" % problems.count())

solutions_obj = Solution.objects.filter(problem__in=problems, ignore=False).order_by('id')
# solutions_obj = Solution.objects.all().order_by('id')
print("Solutions to be used: %d" % solutions_obj.count())

docs_id = []
questions = []
solutions = []

# Fill separated structures
for sol in solutions_obj:
    docs_id.append(sol.id)
    questions.append(sol.problem.content)
    solutions.append(sol.content)

print("Got %d documents" %(solutions_obj.count()))

Problems to be ignored: 656
Problems to be used: 132
Solutions to be used: 54
Got 54 documents


In [147]:
concepts = SolutionConcept.objects.all()
concepts_idx = list(Concept.objects.all().values_list('pk', flat=True).order_by('pk'))
users_idx = list(concepts.values_list('user__pk', flat=True).distinct())

In [114]:
import statsmodels.stats.inter_rater as irr

In [149]:
# rater_matrix = np.zeros((len(users_idx), len(concepts_idx)*len(solutions_obj)))
# rater_matrix = np.zeros((len(users_idx), len(concepts_idx)))

k_alpha_py = []
for idx_solution, sol in enumerate(solutions_obj): 
    concept_solution = concepts.filter(solution=sol).values('user__pk', 'concept__pk')
    users_idx = list(concept_solution.values_list('user__pk', flat=True).distinct())
#     concepts_idx = list(concept_solution.values_list('concept__pk', flat=True).distinct().order_by('concept__pk'))
    rater_matrix = np.zeros((len(users_idx), len(concepts_idx)))
    
    for pair in concept_solution:
        i = users_idx.index(pair['user__pk'])
#         j = idx_solution * len(concepts_idx) + concepts_idx.index(pair['concept__pk'])
        j = concepts_idx.index(pair['concept__pk'])
        rater_matrix[i, j] = 1

#     rater_masked = ma.masked_values(rater_matrix, 0)
    # Saving to use in R
    np.savetxt("%s/rater_matrix_%d.csv" % (folder, idx_solution), rater_matrix, delimiter=",")
    # Calculate it in Python
    k_alpha = krippendorff_alpha(rater_matrix, nominal_metric)
    k_alpha_py.append(k_alpha)
    print("%d - %.2f" % (idx_solution, k_alpha))
    
np.mean(k_alpha_py)
np.std(k_alpha_py)

0 - 0.59
1 - 0.53
2 - 0.32
3 - 0.45
4 - 0.66
5 - 0.52
6 - 0.52
7 - 0.39
8 - 0.36
9 - 0.52
10 - 0.45
11 - 0.23
12 - 0.31
13 - 0.59
14 - 0.52
15 - 0.63
16 - 0.52
17 - 0.32
18 - 0.58
19 - 0.25
20 - 0.59
21 - 0.45
22 - 0.32
23 - 0.73
24 - 0.40
25 - 0.21
26 - 0.66
27 - 0.45
28 - 0.19
29 - 0.52
30 - 0.52
31 - 0.39
32 - 0.32
33 - 0.39
34 - 0.45
35 - 0.59
36 - 0.68
37 - 0.39
38 - 0.38
39 - 0.31
40 - 0.18
41 - 0.32
42 - 0.16
43 - 0.20
44 - 0.31
45 - 0.44
46 - 0.31
47 - 0.39
48 - 0.37
49 - -0.02
50 - 0.19
51 - 0.39
52 - 0.33
53 - 0.45


0.1532194502158611

In [152]:
np.median(k_alpha_py)
np.std(k_alpha_py)

0.1532194502158611

In [ ]:
irr.fleiss_kappa

In [ ]:
%%R
library(irr)

In [151]:
%%R

data <- read.csv('data/tese/rater_matrix_49.csv', header=FALSE)
# data <- replace(data, data == 0, NA)
nmm <- as.matrix(data)

# first assume the default nominal classification
kripp.alpha(nmm)

 Krippendorff's alpha

 Subjects = 15 
   Raters = 4 
    alpha = -0.0359 


In [123]:
rater_matrix.sum(axis=0)

array([4., 0., 0., 0., 0., 1., 0., 3., 0., 0., 0., 0., 0., 3., 1.])

In [115]:
rater_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [87]:
#! /usr/bin/env python
# -*- coding: utf-8
'''
Python implementation of Krippendorff's alpha -- inter-rater reliability
(c)2011-17 Thomas Grill (http://grrrr.org)
Python version >= 2.4 required
'''

from __future__ import print_function
try:
    import numpy as np
except ImportError:
    np = None


def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2


def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or 
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.

In [104]:
array = rater_masked

print("nominal metric: %.3f" % krippendorff_alpha(array, nominal_metric, missing_items=missing))
print("interval metric: %.3f" % krippendorff_alpha(array, interval_metric, missing_items=missing))

nominal metric: 1.000
interval metric: 1.000
